In [6]:
import pandas as pd

DATA_PATH = r"C:\DineSense AI\data\raw\Dataset .csv"
df = pd.read_csv(DATA_PATH)
df.head()


,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [9]:
print("Shape:", df.shape)
print("\nColumns:\n", df.columns.tolist())

# ------------------------------
# 2. Check for Missing Values
# ------------------------------
print("\nMissing Values per Column:")
print(df.isnull().sum())

# ------------------------------
# 3. Check Duplicates (Safe)
# ------------------------------
# Use restaurant_id if exists, else skip
if 'restaurant_id' not in df.columns:
    print("\n⚠ 'restaurant_id' column not found. Creating temporary ID column for duplicates check.")
    df['restaurant_id'] = range(1, len(df) + 1)

duplicates = df.duplicated(subset='restaurant_id').sum()
print(f"\nDuplicate restaurant_id rows: {duplicates}")

# ------------------------------
# 4. Check Binary Columns
# ------------------------------
binary_cols = ["has_table_booking", "has_online_delivery", "is_delivering_now", "switch_to_order_menu"]
for col in binary_cols:
    if col in df.columns:
        print(f"\n{col} unique values: {df[col].unique()}")

# ------------------------------
# 5. Check Rating Column
# ------------------------------
if 'aggregate_rating' in df.columns:
    print("\nAggregate rating stats:")
    print(df['aggregate_rating'].describe())
    print("Rating Range:", df['aggregate_rating'].min(), "to", df['aggregate_rating'].max())
else:
    print("\n⚠ 'aggregate_rating' column not found!")

# ------------------------------
# 6. Check Cuisine Column
# ------------------------------
if 'cuisines' in df.columns:
    print("\nSample cuisines (first 10 rows):")
    print(df['cuisines'].head(10))
else:
    print("\n⚠ 'cuisines' column not found!")

# ------------------------------
# 7. Check Latitude and Longitude
# ------------------------------
lat_issue = long_issue = 0
if 'latitude' in df.columns:
    lat_issue = df[~df['latitude'].between(-90, 90)].shape[0]
if 'longitude' in df.columns:
    long_issue = df[~df['longitude'].between(-180, 180)].shape[0]

print(f"\nLatitude out-of-range values: {lat_issue}")
print(f"Longitude out-of-range values: {long_issue}")

# ------------------------------
# 8. Quick Stats for Numeric Columns
# ------------------------------
print("\nNumeric Column Summary:")
print(df.describe())

Shape: (9551, 22)

Columns:
 ['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address', 'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines', 'Average Cost for two', 'Currency', 'Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu', 'Price range', 'Aggregate rating', 'Rating color', 'Rating text', 'Votes', 'restaurant_id']

Missing Values per Column:
Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
restaurant_id           0
dtype: int64


In [12]:
# ------------------------------
# 2. Standardize Column Names
# ------------------------------
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print("Columns after standardization:", df.columns.tolist())

# ------------------------------
# 3. Handle Missing Values
# ------------------------------

# Categorical columns
cat_cols = [
    'restaurant_name', 'city', 'address', 'locality', 'locality_verbose',
    'cuisines', 'currency', 'rating_color', 'rating_text'
]
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].fillna('unknown')

# Numerical columns
num_cols = [
    'average_cost_for_two', 'longitude', 'latitude', 'votes', 'aggregate_rating'
]
for col in num_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# ------------------------------
# 4. Handle Duplicates
# ------------------------------
if 'restaurant_id' not in df.columns:
    df['restaurant_id'] = range(1, len(df) + 1)

df = df.drop_duplicates(subset='restaurant_id')
print("Shape after removing duplicates:", df.shape)

# ------------------------------
# 5. Handle Binary Columns (Yes/No → 1/0)
# ------------------------------
binary_cols = ['has_table_booking', 'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu']
binary_map = {"Yes": 1, "No": 0}

for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].map(binary_map).fillna(0).astype(int)
        print(f"{col} unique values:", df[col].unique())

# ------------------------------
# 6. Validate Ratings
# ------------------------------
if 'aggregate_rating' in df.columns:
    df = df[df['aggregate_rating'] > 0]  # remove invalid ratings
    print("Rating range after cleaning:", df['aggregate_rating'].min(), "to", df['aggregate_rating'].max())

# ------------------------------
# 7. Clean Cuisines
# ------------------------------
if 'cuisines' in df.columns:
    # Convert everything to string
    df['cuisines'] = df['cuisines'].astype(str)
    
    # Replace 'nan' strings with 'unknown'
    df['cuisines'] = df['cuisines'].replace('nan', 'unknown')
    
    # Lowercase, strip, split by comma
    df['cuisines'] = df['cuisines'].str.lower().str.strip().str.split(", ")
    
    print("Sample cuisines after cleaning:", df['cuisines'].head(5))


# ------------------------------
# 8. Validate Latitude & Longitude
# ------------------------------
if 'latitude' in df.columns and 'longitude' in df.columns:
    df = df[df['latitude'].between(-90, 90)]
    df = df[df['longitude'].between(-180, 180)]
    print("Shape after validating coordinates:", df.shape)

# ------------------------------
# 9. Handle Outliers (Average Cost)
# ------------------------------
if 'average_cost_for_two' in df.columns:
    upper_limit = df['average_cost_for_two'].quantile(0.99)
    df = df[df['average_cost_for_two'] <= upper_limit]
    print("Shape after removing cost outliers:", df.shape)

# ------------------------------
# 10. Reset Index
# ------------------------------
df = df.reset_index(drop=True)


Columns after standardization: ['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address', 'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines', 'average_cost_for_two', 'currency', 'has_table_booking', 'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu', 'price_range', 'aggregate_rating', 'rating_color', 'rating_text', 'votes', 'restaurant_id']
Shape after removing duplicates: (7340, 22)
has_table_booking unique values: [0]
has_online_delivery unique values: [0]
is_delivering_now unique values: [0]
switch_to_order_menu unique values: [0]
Rating range after cleaning: 1.8 to 4.9
Sample cuisines after cleaning: 0            [['french', 'japanese', 'desserts']]
1                                  [['japanese']]
2    [['seafood', 'asian', 'filipino', 'indian']]
3                         [['japanese', 'sushi']]
4                        [['japanese', 'korean']]
Name: cuisines, dtype: object
Shape after validating coordinates: (7340, 22)
Shape after rem

In [13]:
# 1. Drop duplicate restaurant_id column
df = df.loc[:, ~df.columns.duplicated()]

# 2. Fix cuisines nested lists
def flatten_cuisine_list(x):
    if isinstance(x, list):
        # Flatten one level
        if len(x) == 1 and isinstance(x[0], list):
            return x[0]
        return x
    else:
        return [str(x)]

df['cuisines'] = df['cuisines'].apply(flatten_cuisine_list)

# Check result
print(df['cuisines'].head())


0    [['french', 'japanese', 'desserts']]
1                          [['japanese']]
2                 [['japanese', 'sushi']]
3                [['japanese', 'korean']]
4                           [['chinese']]
Name: cuisines, dtype: object


In [14]:
binary_cols = ['has_table_booking', 'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu']
df = df.drop(columns=binary_cols)


In [15]:
print("\nDataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())

# ------------------------------
# 2. Check for Missing Values
# ------------------------------
missing = df.isnull().sum()
print("\nMissing values per column:")
print(missing)

# ------------------------------
# 3. Check for Duplicates
# ------------------------------
if 'restaurant_id' in df.columns:
    dup_count = df.duplicated(subset='restaurant_id').sum()
    print(f"\nDuplicate restaurant_id rows: {dup_count}")
else:
    print("\n⚠ 'restaurant_id' column missing!")

# ------------------------------
# 4. Check Binary Columns
# ------------------------------
binary_cols = ['has_table_booking', 'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu']
for col in binary_cols:
    if col in df.columns:
        print(f"{col} unique values: {df[col].unique()}")

# ------------------------------
# 5. Check Ratings
# ------------------------------
if 'aggregate_rating' in df.columns:
    print("\nAggregate rating stats:")
    print(df['aggregate_rating'].describe())
    print("Rating range:", df['aggregate_rating'].min(), "to", df['aggregate_rating'].max())

# ------------------------------
# 6. Check Cuisines Column
# ------------------------------
if 'cuisines' in df.columns:
    # Check for nested lists
    nested_count = df['cuisines'].apply(lambda x: isinstance(x, str) and x.startswith('[[')).sum()
    print(f"\nCuisines nested list rows: {nested_count} (should be 0)")
    
    # Show sample
    print("\nSample cuisines:")
    print(df['cuisines'].head(10))
else:
    print("\n⚠ 'cuisines' column missing!")

# ------------------------------
# 7. Check Latitude & Longitude
# ------------------------------
lat_issue = long_issue = 0
if 'latitude' in df.columns:
    lat_issue = df[~df['latitude'].between(-90, 90)].shape[0]
if 'longitude' in df.columns:
    long_issue = df[~df['longitude'].between(-180, 180)].shape[0]
print(f"\nLatitude out-of-range values: {lat_issue}")
print(f"Longitude out-of-range values: {long_issue}")

# ------------------------------
# 8. Numeric Column Summary
# ------------------------------
print("\nNumeric column summary:")
print(df.describe())

# ------------------------------
# 9. Final Cleanliness Check
# ------------------------------
issues = missing.sum() + dup_count + nested_count + lat_issue + long_issue
if issues == 0:
    print("\n✅ Data is fully cleaned and ready for modeling!")
else:
    print("\n⚠ Data has some remaining issues. Please review above warnings.")


Dataset shape: (7299, 17)

Columns: ['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address', 'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines', 'average_cost_for_two', 'currency', 'price_range', 'aggregate_rating', 'rating_color', 'rating_text', 'votes']

Missing values per column:
restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
dtype: int64

Duplicate restaurant_id rows: 0

Aggregate rating stats:
count    7299.000000
mean        3.432936
std         0.550448
min         1.800000
25%         3.000000
50%         3.400000
75%         3.800000
max         4.900000
Na

In [16]:
import os

# ------------------------------
# 1. Define save path
# ------------------------------
SAVE_DIR = r"C:\DineSense AI\data\processed"
SAVE_PATH = os.path.join(SAVE_DIR, "clean_restaurants.csv")

# ------------------------------
# 2. Create folder if it doesn't exist
# ------------------------------
os.makedirs(SAVE_DIR, exist_ok=True)

# ------------------------------
# 3. Save the cleaned dataset
# ------------------------------
df.to_csv(SAVE_PATH, index=False)
print(f"✅ Clean dataset saved at: {SAVE_PATH}")


✅ Clean dataset saved at: C:\DineSense AI\data\processed\clean_restaurants.csv
